In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The Data

In [3]:
#df = pd.read_csv('../../data/more_feats_0.2.csv')
#df = pd.read_csv('../../data/more_features/more_feats_0.3.csv')
df= pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df = df.fillna(-1)
#df = utils.balance_dataset(df, 1200)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,11.183192,-1.000000,-1.000000,1.661383,-1.000000,95.006424,-1.000000,6.758433,60.586525,1,2.976104,-1.000000,-1.0,-1.000000,-1.000000,49.530542,-1.000000,-1.0,Inconclusive diagnosis
1,14.387445,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,6.350652,86.964793,1,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,No anemia
2,12.749357,-1.000000,-1.000000,-1.000000,-1.000000,76.759285,159.834784,5.018156,67.065069,1,2.705719,442.293823,-1.0,56.850479,75.739552,72.072041,20.600875,-1.0,Inconclusive diagnosis
3,11.508870,197.180945,1.200125,0.000000,457.033309,102.900301,131.177927,6.656823,66.403181,1,1.869338,223.896404,-1.0,111.220307,66.999185,18.353272,14.132423,-1.0,Unspecified anemia
4,9.456656,427.952052,-1.000000,0.660252,-1.000000,104.543774,-1.000000,-1.000000,-1.000000,0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,Vitamin B12/Folate deficiency anemia


In [4]:
df.label.value_counts()

Inconclusive diagnosis                  15544
No anemia                               10000
Anemia of chronic disease                8792
Iron deficiency anemia                   7624
Unspecified anemia                       7055
Aplastic anemia                          7036
Hemolytic anemia                         7001
Vitamin B12/Folate deficiency anemia     6948
Name: label, dtype: int64

In [5]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((49000, 18), (21000, 18), (49000,), (21000,))

In [6]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'age',
 'gender',
 'indirect_bilirubin',
 'transferrin',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose']

In [7]:
len(action_list)

26

#### Training 

In [8]:
# %%time
# timesteps = int(1e6)
# dqn_model = utils.stable_dqn(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [101]:
for steps in [int(4e6)]: #I am using the old environment !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#for steps in [int(2e3)]:
#     start_time = datetime.now()
    dqn_model = utils.stable_dqn(X_train, y_train, steps, True, 
                                 f'../../models/many_features/0.1/dqn_by_type_new_labels_{steps}')
#     end_time = datetime.now()
#     print(f'The duration for {steps} steps is {end_time-start_time}')

using just stable baselines (not 3)
--------------------------------------
| % time spent exploring  | 92       |
| episodes                | 10000    |
| mean 100 episode reward | -1       |
| steps                   | 32327    |
| success rate            | 0.11     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 83       |
| episodes                | 20000    |
| mean 100 episode reward | -1.2     |
| steps                   | 67632    |
| success rate            | 0.09     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 73       |
| episodes                | 30000    |
| mean 100 episode reward | -1.2     |
| steps                   | 106434   |
| success rate            | 0.15     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 63       |
| episodes                | 40000    |
| mean 100 episode reward | 

--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 310000   |
| mean 100 episode reward | -7.5     |
| steps                   | 2592748  |
| success rate            | 0.19     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 320000   |
| mean 100 episode reward | -7.2     |
| steps                   | 2688528  |
| success rate            | 0.26     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 330000   |
| mean 100 episode reward | -7.9     |
| steps                   | 2784340  |
| success rate            | 0.25     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 340000   |
| mean 100 episode reward | -7.4     |
| steps                  

#### Testing

In [102]:
training_env = utils.create_env(X_train, y_train)
dqn_model = utils.load_dqn('../../models/many_features/0.1/dqn_by_type_new_labels_4000000.pkl', training_env)

Using just stable baselines (not 3)


In [103]:
test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)

Count: 4200
Count: 8400
Count: 12600
Count: 16800
Count: 21000
Testing done.....


In [104]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

22.204761904761906

In [105]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(10.0, -8.072333333333333)

In [106]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.22204761904761905, 0.04542532050033122, 0.5)

In [107]:
test_df.y_pred.unique()

array([7.])

In [108]:
test_df.y_pred.value_counts()

7.0    21000
Name: y_pred, dtype: int64

In [109]:
test_df[test_df.y_pred == 7]

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,10.0,0.0,0.0,-8.0,1.0,"[mcv, gender, gender, gender, gender, gender, ...",6.0,7.0
1,10.0,1.0,0.0,-8.0,1.0,"[mcv, gender, gender, gender, gender, gender, ...",3.0,7.0
2,10.0,2.0,0.0,-8.0,1.0,"[mcv, gender, mcv, mcv, mcv, mcv, mcv, mcv, mc...",2.0,7.0
3,10.0,3.0,1.0,-8.0,1.0,"[mcv, gender, mcv, mcv, mcv, mcv, mcv, mcv, mc...",7.0,7.0
4,10.0,4.0,0.0,-8.0,1.0,"[mcv, gender, mcv, mcv, mcv, mcv, mcv, mcv, mc...",6.0,7.0
...,...,...,...,...,...,...,...,...
20995,10.0,20995.0,1.0,-8.0,1.0,"[mcv, gender, gender, gender, gender, gender, ...",7.0,7.0
20996,10.0,20996.0,0.0,-8.0,1.0,"[mcv, gender, gender, gender, gender, gender, ...",4.0,7.0
20997,10.0,20997.0,0.0,-8.0,1.0,"[mcv, gender, mcv, mcv, mcv, mcv, mcv, mcv, mc...",5.0,7.0
20998,10.0,20998.0,0.0,-8.0,1.0,"[mcv, gender, mcv, mcv, mcv, mcv, mcv, mcv, mc...",2.0,7.0


#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/test_df_2800000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/success_df_2800000.csv', index=False)